In [10]:
!pip install gluonts
!pip install --upgrade datasets
!pip install utilsforecast

In [11]:
!git clone https://github.com/GiuliaGhisolfi/TSFM-ZeroShotEval
%cd TSFM-ZeroShotEval/src

c:\Documenti\VS_Code\tesi\TSFM-ZeroShotEval\src\TSFM-ZeroShotEval\src\TSFM-ZeroShotEval\src


Cloning into 'TSFM-ZeroShotEval'...
Updating files:  48% (73/151)
Updating files:  49% (74/151)
Updating files:  50% (76/151)
Updating files:  51% (78/151)
Updating files:  52% (79/151)
Updating files:  53% (81/151)
Updating files:  54% (82/151)
Updating files:  55% (84/151)
Updating files:  56% (85/151)
Updating files:  57% (87/151)
Updating files:  58% (88/151)
Updating files:  59% (90/151)
Updating files:  60% (91/151)
Updating files:  61% (93/151)
Updating files:  62% (94/151)
Updating files:  63% (96/151)
Updating files:  64% (97/151)
Updating files:  65% (99/151)
Updating files:  66% (100/151)
Updating files:  67% (102/151)
Updating files:  68% (103/151)
Updating files:  69% (105/151)
Updating files:  70% (106/151)
Updating files:  71% (108/151)
Updating files:  72% (109/151)
Updating files:  73% (111/151)
Updating files:  74% (112/151)
Updating files:  75% (114/151)
Updating files:  76% (115/151)
Updating files:  77% (117/151)
Updating files:  78% (118/151)
Updating files:  79% 

### Import data

In [12]:
import json
with open("data/chronos_data_proprieties.json") as f:
    dataset_properties_map = json.load(f)

In [13]:
DATASET_CHRONOS = ["exchange_rate", "ercot", "dominick"]

### Results file

In [14]:
import csv
import os

# Ensure the output directory exists
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "chronos_data_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
        ]
    )

## Evaluation

In [15]:
from gluonts.ev.metrics import (
    MSE,
    MAE,
    MASE,
    MAPE,
    SMAPE,
    MSIS,
    RMSE,
    NRMSE,
    ND,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(
        quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ),
]

### Chronos

In [16]:
model_name = "chronos_bolt_small" # TODO: change to "chronos_t5_base" for the original Chronos model

model_path="amazon/chronos-bolt-small"
# TODO: use "amazon/chronos-t5-base" for the corresponding original Chronos model
# "amazon/chronos-bolt-tiny", "amazon/chronos-bolt-mini", "amazon/chronos-bolt-small", "amazon/chronos-bolt-base",
# "amazon/chronos-t5-tiny", "amazon/chronos-t5-mini", "amazon/chronos-t5-small",
# "amazon/chronos-t5-base", "amazon/chronos-t5-large",

In [ ]:
from utils.chronos_predictor import ChronosPredictor
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
import pandas as pd


for ds_name in [DATASET_CHRONOS[0]]:
        print(f"Processing dataset: {ds_name}")
        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            ########################################################################################
            import pandas as pd
            import numpy as np
            from gluonts.dataset.split import TestData, OffsetSplitter
            from gluonts.dataset.common import ListDataset, FieldName

            df = pd.read_parquet("data/chronos_benchmark/exchange_rate.arrow")
            dataset_gluonts = ListDataset(
                [
                    {
                        FieldName.ITEM_ID: index,
                        FieldName.START: pd.Timestamp(row['start']),
                        FieldName.TARGET: np.asarray(row['target'], dtype=np.float32)
                    }
                    for index, row in df.iterrows()
                ],
                freq="D" #dataset_properties_map[ds_name]["freq"]
            )
            dataset = TestData( # TODO: change
                dataset=dataset_gluonts,
                splitter=OffsetSplitter(offset=-8),
                prediction_length=8,
                windows=15,
                distance=None,
                max_history=None
            )
            ########################################################################################

            season_length = get_seasonality(dataset_properties_map[ds_name]["freq"])
            prediction_length = dataset_properties_map[ds_name]["prediction_lengths"]

            predictor = ChronosPredictor(
                model_path=model_path,
                num_samples=20,
                prediction_length=prediction_length,
                # Change device_map to "cpu" to run on CPU or "cuda" to run on GPU
                device_map="cuda",
            )

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            with open(csv_file_path, "a", newline="") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(
                    [
                        ds_name,
                        model_name,
                        res["MSE[mean]"][0],
                        res["MSE[0.5]"][0],
                        res["MAE[0.5]"][0],
                        res["MASE[0.5]"][0],
                        res["MAPE[0.5]"][0],
                        res["sMAPE[0.5]"][0],
                        res["MSIS"][0],
                        res["RMSE[mean]"][0],
                        res["NRMSE[mean]"][0],
                        res["ND[0.5]"][0],
                        res["mean_weighted_sum_quantile_loss"][0],
                        ds_name,
                        dataset_properties_map[ds_name]["num_variates"],
                    ]
                )

        print(f"Results for {ds_name} have been written to {csv_file_path}")

In [ ]:
# TODO: delete

import pandas as pd
import numpy as np
from gluonts.dataset.split import TestData, OffsetSplitter
from gluonts.dataset.common import ListDataset, FieldName

df = pd.read_parquet("data/chronos_benchmark/exchange_rate.arrow")
dataset_gluonts = ListDataset(
    [
        {
            FieldName.ITEM_ID: index,
            FieldName.START: pd.Timestamp(row['start']),
            FieldName.TARGET: np.asarray(row['target'], dtype=np.float32)
        }
        for index, row in df.iterrows()
    ],
    freq="D" #dataset_properties_map[ds_name]["freq"]
)
dataset = TestData( # TODO: change
    dataset=dataset_gluonts,
    splitter=OffsetSplitter(offset=-8),
    prediction_length=8,
    windows=15,
    distance=None,
    max_history=None
)

dataset

TestData(dataset=[{'item_id': 0, 'start': Period('1990-01-01', 'D'), 'target': array([0.7855  , 0.7818  , 0.7867  , ..., 0.723197, 0.720825, 0.720825],
      dtype=float32)}, {'item_id': 1, 'start': Period('1990-01-01', 'D'), 'target': array([1.611   , 1.61    , 1.6293  , ..., 1.234111, 1.233905, 1.233905],
      dtype=float32)}, {'item_id': 2, 'start': Period('1990-01-01', 'D'), 'target': array([0.861698, 0.861104, 0.86103 , ..., 0.745184, 0.744131, 0.744131],
      dtype=float32)}, {'item_id': 3, 'start': Period('1990-01-01', 'D'), 'target': array([0.634196, 0.633513, 0.648508, ..., 0.984446, 0.980344, 0.980344],
      dtype=float32)}, {'item_id': 4, 'start': Period('1990-01-01', 'D'), 'target': array([0.211242, 0.211242, 0.211242, ..., 0.143997, 0.143993, 0.143993],
      dtype=float32)}, {'item_id': 5, 'start': Period('1990-01-01', 'D'), 'target': array([0.006838, 0.006863, 0.006975, ..., 0.008562, 0.008555, 0.008555],
      dtype=float32)}, {'item_id': 6, 'start': Period('1990-01-